In [1]:

from torch.utils.data import DataLoader, TensorDataset
import torch
import numpy as np
import matplotlib.pyplot as plt
#load model
import torchvision
from torchvision import transforms

from h5_utils import h5read, h5disp

In [2]:
images = h5read("../symmetry_project/dataset/dataset.h5", "images")
obj =  h5read("../symmetry_project/dataset/dataset.h5", "obj")
ost =  h5read("../symmetry_project/dataset/dataset.h5", "ost")

In [ ]:
import os
from torchvision.utils import save_image
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

# Create output directory
os.makedirs('muri1320', exist_ok=True)

# Assuming dataset_transformed is a TensorDataset of shape [N, 3, 224, 224]
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=1, shuffle=False)

# Optional: reverse any normalization (update if you used custom preprocessing)
unnormalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)

for i, (img_tensor,) in enumerate(data_loader):
    img = unnormalize(img_tensor.squeeze(0))  # remove batch dimension and unnormalize
    img = torch.clamp(img, 0, 1)  # clip to [0, 1] range
    save_image(img, f'muri1320/image_{i:04d}.png')  # save with zero-padded index


In [3]:
images.shape

(1320, 3, 256, 256)

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize the image to 224x224 (input size for AlexNet)
    transforms.Lambda(lambda x: (x / 255.0)),  # Rescale by dividing by 255
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

dataset_tensor = torch.tensor(images, dtype=torch.float32)
dataset_transformed = torch.stack([transform(img) for img in dataset_tensor])  # Apply transform to each image

In [4]:
# load model
import timm
model = timm.create_model("efficientnet_b0", pretrained=True)
model = model.eval()

In [5]:
model

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2

In [6]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.blocks[6][0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
#np.save('./model_features/efficientnet_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 320, 7, 7]) tensor(-33.5975) tensor(35.0662)


In [ ]:
output_features

[]

In [19]:
model = torchvision.models.resnet18(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [20]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer4[0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet18_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 512, 7, 7]) tensor(0.) tensor(6.3313)


In [21]:
output_features.reshape(1320, -1).shape

torch.Size([1320, 25088])

In [22]:
from collections import OrderedDict
# Load checkpoint
checkpoint = torch.load("model_features/resnet18_l2_eps3.ckpt", map_location="cpu")

# Extract the nested state_dict
state_dict = checkpoint["model"]

# Remove the prefix: "module.model.model."
clean_state_dict = OrderedDict()
prefix = "module.model."
for k, v in state_dict.items():
    if k.startswith(prefix):
        new_k = k[len(prefix):]
        clean_state_dict[new_k] = v

# Try loading weights
model.load_state_dict(clean_state_dict)

<All keys matched successfully>

In [23]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer4[0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet18_robust_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 512, 7, 7]) tensor(0.) tensor(7.6611)


In [24]:
import timm
model = timm.create_model('resnet18.fb_ssl_yfcc100m_ft_in1k', pretrained=True)
model = model.eval()

In [25]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer4[0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet18_ssl_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 512, 7, 7]) tensor(0.) tensor(173.6028)


In [4]:
model = torchvision.models.resnet50(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer4[0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet_features.npy', output_features.numpy())

In [62]:
import timm
model = timm.create_model('resnet50.fb_swsl_ig1b_ft_in1k', pretrained=True)
model = model.eval()


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [63]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer4[0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet50_swsl_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 2048, 7, 7]) tensor(0.) tensor(211.0662)


In [64]:
model = torch.hub.load('facebookresearch/dino:main', 'dino_resnet50')
model = model.eval()

Using cache found in /users/smuzelle/.cache/torch/hub/facebookresearch_dino_main
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [65]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer4[0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet50_ssl_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 2048, 7, 7]) tensor(0.) tensor(18.5285)


In [ ]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer4[0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet50_swsl_features.npy', output_features.numpy())

In [4]:
model = torchvision.models.resnet152(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer3[34].bn3.register_forward_hook(hook_fn) #layer3[35]

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet152_2_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 1024, 14, 14]) tensor(0.) tensor(5.7675)


In [12]:
model = torchvision.models.resnet101(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /users/smuzelle/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 115MB/s] 


In [13]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer3[22].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet101_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 1024, 14, 14]) tensor(0.) tensor(15.1838)


In [92]:
model = timm.create_model('resnext101_32x4d.fb_ssl_yfcc100m_ft_in1k', pretrained=True)
model = model.eval()

model.safetensors:   0%|          | 0.00/177M [00:00<?, ?B/s]

In [93]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer3[22].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet101_ssl_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 1024, 14, 14]) tensor(0.) tensor(3.8273)


In [17]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-small-patch4-window8-256")
model = AutoModelForImageClassification.from_pretrained("microsoft/swinv2-small-patch4-window8-256")


preprocessor_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.9k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/199M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/199M [00:00<?, ?B/s]

In [18]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.swinv2.encoder.layers[3].blocks[0].layernorm_after.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/swin_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 49, 768]) tensor(-114.8137) tensor(10.2390)


In [8]:
from transformers import AutoImageProcessor, Swinv2ForMaskedImageModeling

processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-small-patch4-window16-256")
model = Swinv2ForMaskedImageModeling.from_pretrained(
    "microsoft/swinv2-small-patch4-window16-256"
)


preprocessor_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.9k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/199M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/199M [00:00<?, ?B/s]

Some weights of Swinv2ForMaskedImageModeling were not initialized from the model checkpoint at microsoft/swinv2-small-patch4-window16-256 and are newly initialized: ['decoder.0.bias', 'decoder.0.weight', 'swinv2.embeddings.mask_token']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.swinv2.encoder.layers[3].blocks[0].layernorm_after.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/swin_ssl_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 49, 768]) tensor(-11.7128) tensor(91.1493)


In [ ]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.layer3[22].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet101_ssl_features.npy', output_features.numpy())

In [5]:
model = torchvision.models.alexnet(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [13]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.features[12].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/alexnet_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([2640, 256, 6, 6]) tensor(0.) tensor(29.3822)


In [26]:
model = torchvision.models.densenet201(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.features.transition3.pool.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/densenet_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 896, 7, 7]) tensor(-0.8038) tensor(2.5911)


In [28]:
model = torchvision.models.densenet169(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.features.transition3.pool.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/densenet169_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 640, 7, 7]) tensor(-0.7527) tensor(1.9534)


In [30]:
model = torchvision.models.densenet121(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [50]:
model = torchvision.models.densenet161(pretrained=True)
model = model.eval()

In [53]:
import torch
from torchvision.models import densenet161
from collections import OrderedDict

# Load checkpoint
checkpoint = torch.load("model_features/densenet_l2_eps3.ckpt", map_location="cpu")

# Extract the nested state_dict
state_dict = checkpoint["model"]

# Remove the prefix: "module.model.model."
clean_state_dict = OrderedDict()
prefix = "module.model.model."
for k, v in state_dict.items():
    if k.startswith(prefix):
        new_k = k[len(prefix):]
        clean_state_dict[new_k] = v

# Create DenseNet model
model = densenet161()  # or 169/201/161 depending on match

# Try loading weights
model.load_state_dict(clean_state_dict)

/tmp/ipykernel_1538819/3532106386.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("model_features/densenet_l2_eps3.ckpt", map_location="cpu")


<All keys matched successfully>

In [52]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.features.transition3.pool.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/densenet161_robust_eps3_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 1056, 7, 7]) tensor(-1.3439) tensor(2.0056)


In [27]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.features.transition3.pool.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/densenet121_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 512, 7, 7]) tensor(-0.9936) tensor(1.8007)


In [13]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=True)
model.eval()

Using cache found in /users/smuzelle/.cache/torch/hub/pytorch_vision_v0.10.0


ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_

In [14]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.stage4.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/shufflenet_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 464, 7, 7]) tensor(0.) tensor(4.7454)


In [15]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=True)
model.eval()

from torchvision.models import densenet161
from collections import OrderedDict

# Load checkpoint
checkpoint = torch.load("model_features/shufflenet_l2_eps3.ckpt", map_location="cpu")

# Extract the nested state_dict
state_dict = checkpoint["model"]

# Remove the prefix: "module.model.model."
clean_state_dict = OrderedDict()
prefix = "module.model.model."
for k, v in state_dict.items():
    if k.startswith(prefix):
        new_k = k[len(prefix):]
        clean_state_dict[new_k] = v

# Try loading weights
model.load_state_dict(clean_state_dict)

Using cache found in /users/smuzelle/.cache/torch/hub/pytorch_vision_v0.10.0
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [16]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.stage4.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/shufflenet_robust_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 464, 7, 7]) tensor(0.) tensor(2.8035)


In [18]:
model = torchvision.models.inception_v3(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.Mixed_7a.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/inceptionv3_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 1280, 5, 5]) tensor(0.) tensor(20.1041)


In [20]:
model = torchvision.models.vit_b_16(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.encoder.layers[6].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/vit_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 197, 768]) tensor(-44.4042) tensor(34.4210)


In [66]:
import timm
model = timm.create_model("vit_small_patch16_224_dino", pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_small_patch16_224_dino to current vit_small_patch16_224.dino.
  model = create_fn(


In [69]:
model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [70]:
deolbatch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.blocks[6].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/vit_ssl_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 197, 384]) tensor(-2.0844) tensor(1.1553)


In [54]:
model = torchvision.models.vgg16_bn(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [55]:
import torch
from collections import OrderedDict

# Load checkpoint
checkpoint = torch.load("model_features/vgg16_bn_l2_eps3.ckpt", map_location="cpu")

# Extract state_dict
state_dict = checkpoint['model']  # this assumes your dict is under 'model'

# Optionally, inspect first few keys
print("Original keys:", list(state_dict.keys())[:5])

# Clean the keys to match the model definition
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith("module.model.model."):
        new_k = k.replace("module.model.model.", "")
    elif k.startswith("module.attacker.model.model."):
        new_k = k.replace("module.attacker.model.model.", "")
    else:
        continue  # Skip anything that doesn't match your intended model
    new_state_dict[new_k] = v

# Load cleaned state_dict into your model
model.load_state_dict(new_state_dict)  # strict=False lets you ignore unmatched keys


/tmp/ipykernel_1538819/2104175007.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("model_features/vgg16_bn_l2_eps3.ckpt", map_location="cpu")


Original keys: ['module.normalizer.new_mean', 'module.normalizer.new_std', 'module.model.model.features.0.weight', 'module.model.model.features.0.bias', 'module.model.model.features.1.weight']


<All keys matched successfully>

In [14]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.features[23].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/vgg16_robust_eps3_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 256, 28, 28]) tensor(0.) tensor(3.6927)


In [7]:
512*14*14

100352

In [5]:
model = torchvision.models.convnext_tiny(pretrained=True)
model = model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

batch_count = 0
output_features = []

def hook_fn(module, input, output):
    print(f"Hook fired at batch {batch_count} with input {input[0].shape}, and output {output.shape}")
    output_features.append(output)

hook = model.features[5][4].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(f"Running batch {batch_count}, shape: {images.shape}")
        _ = model(images)
        batch_count += 1

hook.remove()

print(len(output_features))
output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/convnext_features.npy', output_features.numpy())

Running batch 0, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 0 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 1, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 1 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 2, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 2 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 3, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 3 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 4, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 4 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 5, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 5 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 6, shape: torch.Size([64, 3, 224

In [74]:
model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=True)
model = model.eval()

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

In [76]:
model

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): GlobalResponseNormMlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (grn): GlobalResponseNorm()
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), 

In [78]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

batch_count = 0
output_features = []

def hook_fn(module, input, output):
    print(f"Hook fired at batch {batch_count} with input {input[0].shape}, and output {output.shape}")
    output_features.append(output)

hook = model.stages[2].blocks[4].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(f"Running batch {batch_count}, shape: {images.shape}")
        _ = model(images)
        batch_count += 1

hook.remove()

print(len(output_features))
output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/convnext_ssl_features.npy', output_features.numpy())

Running batch 0, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 0 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 1, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 1 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 2, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 2 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 3, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 3 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 4, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 4 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 5, shape: torch.Size([64, 3, 224, 224])
Hook fired at batch 5 with input torch.Size([64, 384, 14, 14]), and output torch.Size([64, 384, 14, 14])
Running batch 6, shape: torch.Size([64, 3, 224

In [7]:
np.load('./model_features/convnext_features.npy').shape

(1320, 384, 14, 14)

In [14]:
len(output_features), output_features[0].shape, output_features[1].shape

(2640, torch.Size([384, 14, 14]), torch.Size([384, 14, 14]))

In [11]:
384*14*14

75264

In [13]:
75264/500

150.528

In [19]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 37.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.26.2
    Uninstalling huggingface-hub-0.26.2:
      Successfully uninstalled huggingface-hub-0.26.2


In [15]:
from transformers import AutoImageProcessor, AutoModelForImageClassification


In [16]:
processor = AutoImageProcessor.from_pretrained("google/mobilenet_v2_1.4_224")
model = AutoModelForImageClassification.from_pretrained("google/mobilenet_v2_1.4_224", trust_remote_code=True, use_safetensors=True)
model.eval()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(48, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), groups=48, bias=False)
        (normalization): BatchNorm2d(48, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(24, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [56]:
from torchvision.models import mobilenet_v2

model = mobilenet_v2(pretrained=True)

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [57]:
import torch
from collections import OrderedDict

# Load checkpoint
checkpoint = torch.load("model_features/mobilenet_l2_eps3.ckpt", map_location="cpu")

# Extract state_dict
state_dict = checkpoint['model']  # this assumes your dict is under 'model'

# Optionally, inspect first few keys
print("Original keys:", list(state_dict.keys())[:5])

# Clean the keys to match the model definition
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith("module.model.model."):
        new_k = k.replace("module.model.model.", "")
    elif k.startswith("module.attacker.model.model."):
        new_k = k.replace("module.attacker.model.model.", "")
    else:
        continue  # Skip anything that doesn't match your intended model
    new_state_dict[new_k] = v

# Load cleaned state_dict into your model
model.load_state_dict(new_state_dict)  # strict=False lets you ignore unmatched keys


Original keys: ['module.normalizer.new_mean', 'module.normalizer.new_std', 'module.model.model.features.0.0.weight', 'module.model.model.features.0.1.weight', 'module.model.model.features.0.1.bias']


/tmp/ipykernel_1538819/2312309495.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("model_features/mobilenet_l2_eps3.ckpt", map_location="cpu")


<All keys matched successfully>

In [21]:
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [25]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    output_features.append(output)

hook = model.features[12].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/mobilenet_v2_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([64, 3, 224, 224])
torch.Size([40, 3, 224, 224])
torch.Size([1320, 96, 14, 14]) tensor(-5.2174) tensor(5.0877)


In [23]:
94*14*14

18424

In [14]:
!pip install timm

In [16]:
import timm

# Load MobileNetV1 with width multiplier 0.75
model = timm.create_model('mobilenetv1_100.ra4_e3600_r224_in1k', pretrained=True)
model.eval()

model.safetensors:   0%|          | 0.00/17.0M [00:00<?, ?B/s]

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): ReLU6(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU6(inplace=True)
        )
        (aa): Identity()
        (se): Identity()
        (conv_pw): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU6(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1)

In [4]:
import torch
import torchvision
import torchvision.models
from torch.utils import model_zoo

# Helper function to move models to GPU or CPU
def move_model_to_device(model):
    """Move the model to GPU if available, otherwise to CPU."""
    if torch.cuda.is_available():
        print("Using GPU (CUDA)")
        model = torch.nn.DataParallel(model).cuda()
    else:
        print("Using CPU")
        model = torch.nn.DataParallel(model)
    return model
    
def load_model(model_name):
    model_urls = {
        'resnet50_trained_on_SIN': 'https://bitbucket.org/robert_geirhos/texture-vs-shape-pretrained-models/raw/6f41d2e86fc60566f78de64ecff35cc61eb6436f/resnet50_train_60_epochs-c8e5653e.pth.tar',
        'resnet50_trained_on_SIN_and_IN': 'https://bitbucket.org/robert_geirhos/texture-vs-shape-pretrained-models/raw/60b770e128fffcbd8562a3ab3546c1a735432d03/resnet50_train_45_epochs_combined_IN_SF-2a0d100e.pth.tar',
        'resnet50_trained_on_SIN_and_IN_then_finetuned_on_IN': 'https://bitbucket.org/robert_geirhos/texture-vs-shape-pretrained-models/raw/60b770e128fffcbd8562a3ab3546c1a735432d03/resnet50_finetune_60_epochs_lr_decay_after_30_start_resnet50_train_45_epochs_combined_IN_SF-ca06340c.pth.tar',
    }

    if "resnet50" in model_name:
        print("Using the ResNet50 architecture.")
        model = torchvision.models.resnet50(pretrained=False)
        model = move_model_to_device(model)
        checkpoint = model_zoo.load_url(model_urls[model_name], map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

    elif "vgg16" in model_name:
        print("Using the VGG-16 architecture.")
        filepath = "./vgg16_train_60_epochs_lr0.01-6c6fcc9f.pth.tar"
        assert os.path.exists(filepath), "Please download the VGG model manually from https://drive.google.com/drive/folders/1A0vUWyU6fTuc-xWgwQQeBvzbwi6geYQK"
        model = torchvision.models.vgg16(pretrained=False)
        model.features = move_model_to_device(model.features)
        checkpoint = torch.load(filepath, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

    elif "alexnet" in model_name:
        print("Using the AlexNet architecture.")
        filepath = "./alexnet_train_60_epochs_lr0.001-b4aa5238.pth.tar"
        assert os.path.exists(filepath), "Please download the AlexNet model manually from https://drive.google.com/drive/folders/1GnxcR6HUyPfRWAmaXwuiMdAMKlL1shTn"
        model = torchvision.models.alexnet(pretrained=False)
        model.features = move_model_to_device(model.features)
        checkpoint = torch.load(filepath, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

    else:
        raise ValueError("Unknown model architecture.")

    model.load_state_dict(checkpoint["state_dict"])
    return model


In [5]:
import functools
from brainscore_vision.model_helpers.activations.pytorch import load_preprocess_images
from brainscore_vision.model_helpers.activations.pytorch import PytorchWrapper
from brainscore_vision.model_helpers.check_submission import check_models

def get_model(name):
    """
    This method fetches an instance of a base model. The instance has to be callable and return a xarray object,
    containing activations. There exist standard wrapper implementations for common libraries, like pytorch and
    keras. Checkout the examples folder, to see more. For custom implementations check out the implementation of the
    wrappers.
    :param name: the name of the model to fetch
    :return: the model instance
    """
    assert name == 'resnet50-SIN_IN_IN'
    model = load_model("resnet50_trained_on_SIN_and_IN_then_finetuned_on_IN")
    preprocessing = functools.partial(load_preprocess_images, image_size=224)
    wrapper = PytorchWrapper(identifier='name', model=model, preprocessing=preprocessing)
    wrapper.image_size = 224
    return wrapper

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/brainscore_core/metrics/__init__.py:16: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


In [6]:
model = get_model('resnet50-SIN_IN_IN')

Using the ResNet50 architecture.


/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Using CPU


In [7]:
model = model._model.module
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    print(output.shape)
    output_features.append(output)

hook = model.layer4[0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnetSIN_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7

In [48]:
for t in range(len(output_features)):
    print(output_features[t].shape)

torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([64, 512, 7, 7])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 512, 14, 14])
torch.Size([6

In [19]:
from brainscore_vision import load_model
model = load_model("CORnet-S")

  Cloning https://github.com/dicarlolab/CORnet.git to /tmp/pip-install-17lqpbya/cornet_15ce99f77d61474e8b4fd8390efa5c8d


  Running command git clone --filter=blob:none --quiet https://github.com/dicarlolab/CORnet.git /tmp/pip-install-17lqpbya/cornet_15ce99f77d61474e8b4fd8390efa5c8d


  Resolved https://github.com/dicarlolab/CORnet.git to commit d0cc17d4b34ad44dedb01683b70eafd15515adad
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


/cifs/data/tserre_lrs/projects/projects/prj_complex/neural_data/symmetry_project/vision/brainscore_vision/models/cornet_s/model.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental fea

In [14]:
dir(model)

['RecordingTarget',
 'Task',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_identifier',
 '_visual_degrees',
 'activations_model',
 'behavior_model',
 'do_behavior',
 'identifier',
 'layers',
 'look_at',
 'look_at_cached',
 'recording_layers',
 'recording_time_bins',
 'start_recording',
 'start_task',
 'time_mapping',
 'visual_degrees']

In [8]:
model.layers

['V1.output-t0',
 'V2.output-t0',
 'V2.output-t1',
 'V4.output-t0',
 'V4.output-t1',
 'V4.output-t2',
 'V4.output-t3',
 'IT.output-t0',
 'IT.output-t1',
 'decoder.avgpool-t0']

In [22]:
model = model.activations_model

In [10]:
model

Sequential(
  (V1): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (nonlin1): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (nonlin2): ReLU(inplace=True)
    (output): Identity()
  )
  (V2): CORblock_S(
    (conv_input): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (skip): Conv2d(128, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (norm_skip): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (nonlin1): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, k

In [23]:
model.recording = 'IT.output-t0'

In [25]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]  # [batch_size, 3, 224, 224]
        print(images.shape)

        # CORnet model returns an xarray.DataArray
        result = model(images, layers=['IT.output-t0'])
        result_tensor = torch.tensor(result.values)  # convert to tensor

        print(result_tensor.shape)
        output_features.append(result_tensor)


output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/cornetS_2_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])


activations:   0%|          | 0/64 [00:00<?, ?it/s]

AttributeError: 'Tensor' object has no attribute 'read'

In [26]:
from brainscore_vision.stimuli import StimulusSet
from torchvision.transforms.functional import to_pil_image

stimuli = StimulusSet()
stimuli['image_id'] = list(range(len(images)))
stimuli['image'] = [to_pil_image(img) for img in images]
stimuli.image_paths = [None] * len(images)  # required field

# Then call the wrapper
result = model(stimuli, layers=['IT.output-t0'])


ModuleNotFoundError: No module named 'brainscore_vision.stimuli'

In [5]:
import timm

model = timm.create_model('nasnetalarge.tf_in1k', pretrained=True)

In [6]:
model

NASNetALarge(
  (conv0): ConvNormAct(
    (conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNormAct2d(
      96, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): Identity()
    )
  )
  (cell_stem_0): CellStem0(
    (conv_1x1): ActConvBn(
      (act): ReLU()
      (conv): Conv2d(96, 42, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(42, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (comb_iter_0_left): BranchSeparables(
      (act_1): ReLU()
      (separable_1): SeparableConv2d(
        (depthwise_conv2d): Conv2dSame(42, 42, kernel_size=(5, 5), stride=(2, 2), groups=42, bias=False)
        (pointwise_conv2d): Conv2d(42, 42, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (bn_sep_1): BatchNorm2d(42, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act_2): ReLU(inplace=True)
      (separable_2): SeparableConv2d(
       

In [7]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    print(output.shape)
    output_features.append(output)

hook = model.cell_12.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/nasnet_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4032, 7, 7

In [8]:
model = torchvision.models.googlenet(pretrained=True)
model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /users/smuzelle/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 112MB/s] 


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [9]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    print(output.shape)
    output_features.append(output)

hook = model.inception4a.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/inceptionv1_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size([64, 512, 14, 14])
torch.Size([64, 3, 224, 224])
torch.Size

In [10]:

model = timm.create_model('pnasnet5large', pretrained=True)
model.eval()


model.safetensors:   0%|          | 0.00/345M [00:00<?, ?B/s]

PNASNet5Large(
  (conv_0): ConvNormAct(
    (conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNormAct2d(
      96, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): Identity()
    )
  )
  (cell_stem_0): CellStem0(
    (conv_1x1): ActConvBn(
      (act): ReLU()
      (conv): Conv2d(96, 54, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(54, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (comb_iter_0_left): BranchSeparables(
      (act_1): ReLU()
      (separable_1): SeparableConv2d(
        (depthwise_conv2d): Conv2dSame(96, 96, kernel_size=(5, 5), stride=(2, 2), groups=96, bias=False)
        (pointwise_conv2d): Conv2d(96, 54, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (bn_sep_1): BatchNorm2d(54, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act_2): ReLU()
      (separable_2): SeparableConv2d(
        (depthwis

In [11]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    print(output.shape)
    output_features.append(output)

hook = model.cell_8.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/pnasnet_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 4320, 7, 7

In [12]:
model = torchvision.models.squeezenet1_1(pretrained=True)
model.eval()

/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /users/smuzelle/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:00<00:00, 25.5MB/s]


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [13]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    print(output.shape)
    output_features.append(output)

hook = model.features[9].expand3x3_activation.register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/squeezenet_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size([64, 192, 13, 13])
torch.Size([64, 3, 224, 224])
torch.Size

In [20]:
!pip install --upgrade robustbench

In [18]:
!pip install git+https://github.com/fra31/auto-attack


  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-t614agmt
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack /tmp/pip-req-build-t614agmt
  Resolved https://github.com/fra31/auto-attack to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py) ... done
  Created wheel for autoattack: filename=autoattack-0.1-py3-none-any.whl size=36229 sha256=4e26ffd10c1bf74c08dbb8ca1e94bcb61b923f40c9cb6f397cddf0d811006ddf
  Stored in directory: /tmp/pip-ephem-wheel-cache-cpkex6vw/wheels/e1/e8/28/65b2724d4c7740785979eb50bf5e1b3986ead22f6c32a87f8f
Successfully built autoattack


In [22]:
from robustbench.model_zoo.enums import BenchmarkDataset
from robustbench import load_model

model = load_model(
    model_name='Salman2020Do_R50',
    dataset=BenchmarkDataset.imagenet,
    threat_model='Linf'
)

model.eval()

Downloading...
From (original): https://drive.google.com/uc?id=1TmT5oGa1UvVjM3d-XeSj_XmKqBNRUg8r
From (redirected): https://drive.google.com/uc?id=1TmT5oGa1UvVjM3d-XeSj_XmKqBNRUg8r&confirm=t&uuid=70230091-58d1-4956-9bdc-197aa61ba839
To: /files22_lrsresearch/CLPS_Serre_Lab/projects/prj_complex/neural_data/symmetry_idiosyncracy/models/imagenet/Linf/Salman2020Do_R50.pt
100%|██████████| 103M/103M [00:01<00:00, 62.8MB/s] 
/users/smuzelle/.conda/envs/symmetry/lib/python3.11/site-packages/robustbench/utils.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during 

Sequential(
  (normalize): ImageNormalizer(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=Tr

In [25]:
from brainscore_vision.model_helpers.check_submission import check_models
from brainscore_vision.model_helpers.activations.pytorch import PytorchWrapper
from brainscore_vision.model_helpers.activations.pytorch import load_preprocess_images
import torchvision
import functools
import torch
import os
from torchvision.models import resnet50
from brainscore_vision.model_helpers.s3 import load_weight_file


def get_model(name):
    """
    This method fetches an instance of a base model. The instance has to be callable and return a xarray object,
    containing activations. There exist standard wrapper implementations for common libraries, like pytorch and
    keras. Checkout the examples folder, to see more. For custom implementations check out the implementation of the
    wrappers.
    :param name: the name of the model to fetch
    :return: the model instance
    """
    assert name == 'resnet50_robust_l2_eps1'

    model = resnet50()
    weights_path = load_weight_file(bucket="brainscore-storage", folder_name="brainscore-vision/models",
                                relative_path="resnet50_robust_l2_eps1/resnet50_l2_eps1.ckpt",
                                version_id="null",
                                sha1="c75d68b7509f9d3829663ca3b627d4265fa9f588")
    sd = torch.load(weights_path, map_location=torch.device('cpu'))
    sd_processed = {}
    for k, v in sd['model'].items():
        if ('attacker' not in k) and ('model' in k):
            sd_processed[k[13:]] = v
    model.load_state_dict(sd_processed)
    preprocessing = functools.partial(load_preprocess_images, image_size=224)
    wrapper = PytorchWrapper(identifier='resnet50_robust_l2_eps1', model=model, preprocessing=preprocessing)
    wrapper.image_size = 224
    return wrapper

In [27]:
model = get_model('resnet50_robust_l2_eps1')
model = model._model
model.eval()

/tmp/ipykernel_2235464/3699088425.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(weights_path, map_location=torch.device('cpu'))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [28]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    print(output.shape)
    output_features.append(output)

hook = model.layer4[0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet_robust_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7

In [29]:
np.save('./model_features/resnet_robust_eps1_features.npy', output_features.numpy())

In [31]:
from brainscore_vision.model_helpers.check_submission import check_models
from brainscore_vision.model_helpers.activations.pytorch import PytorchWrapper
from brainscore_vision.model_helpers.activations.pytorch import load_preprocess_images
from brainscore_vision.model_helpers.s3 import load_weight_file
import functools
import torch
import os
from torchvision.models import resnet50



def get_model(name):
    """
    This method fetches an instance of a base model. The instance has to be callable and return a xarray object,
    containing activations. There exist standard wrapper implementations for common libraries, like pytorch and
    keras. Checkout the examples folder, to see more. For custom implementations check out the implementation of the
    wrappers.
    :param name: the name of the model to fetch
    :return: the model instance
    """
    assert name == 'resnet50_robust_l2_eps3'

    model = resnet50()
    weights_path = load_weight_file(bucket="brainscore-storage", folder_name="brainscore-vision/models",
                                   relative_path="resnet50_robust_l2_eps3/imagenet_l2_3_0.pt",
                                   version_id="null",
                                   sha1="cc6e4441abc8ad6d2f4da5db84836e544bfb53fd")
    sd = torch.load(weights_path, map_location=torch.device('cpu'))
    sd_processed = {}
    for k, v in sd['model'].items():
        if ('attacker' not in k) and ('model' in k):
            sd_processed[k[13:]] = v 
    model.load_state_dict(sd_processed)
    preprocessing = functools.partial(load_preprocess_images, image_size=224)
    wrapper = PytorchWrapper(identifier='resnet50_robust_l2_eps3', model=model, preprocessing=preprocessing)
    wrapper.image_size = 224
    return wrapper


In [32]:
model = get_model('resnet50_robust_l2_eps3')
model = model._model
model.eval()

brainscore-storage/brainscore-vision/models/resnet50_robust_l2_eps3/imagenet_l2_3_0.pt: 100%|██████████| 205M/205M [00:04<00:00, 48.7MB/s] 
/tmp/ipykernel_2235464/1250554057.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub f

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [33]:
batch_size = 64  # Adjust as needed
data_loader = DataLoader(TensorDataset(dataset_transformed), batch_size=batch_size, shuffle=False)

output_features = []

def hook_fn(module, input, output):
    print(output.shape)
    output_features.append(output)

hook = model.layer4[0].register_forward_hook(hook_fn)

with torch.no_grad():
    for batch in data_loader:
        images = batch[0]
        print(images.shape)
        _ = model(images)
        # break

hook.remove()

output_features = torch.cat(output_features, dim=0)
print(output_features.shape, output_features.min(), output_features.max())
np.save('./model_features/resnet_robust_eps3_features.npy', output_features.numpy())

torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7])
torch.Size([64, 3, 224, 224])
torch.Size([64, 2048, 7, 7